In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project, path_code

from sample.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter
from scipy.ndimage import uniform_filter1d
from scipy import stats

In [2]:
window=800
infos = pickle.load(open(f'{path_code}/MscThesis/Results/straight_bait_{window}.pick', "rb"))

In [3]:
def get_angle(xa,ya,xb,yb):
    dot_product = (xa*xb+ya*yb)/np.sqrt((xa**2+ya**2)*(xb**2+yb**2))
    angle = (np.arccos(dot_product) / (2 * np.pi) * 360)*(1-2*((ya * xb - yb * xa) >= 0))
    return(angle)

In [5]:
infos['abs_angle_to_P']=np.abs(infos['angle_to_P'])
infos['inv_dens']=1/infos['density']
infos['side_cross']=infos.apply (lambda row: comments[row['plate']] if row['plate'] in comments.keys() else 'None', axis=1)
infos['x']=infos['x'].astype(float)
infos['vx']=infos['vx'].astype(float)
infos['y']=infos['y'].astype(float)
infos['vy']=infos['vy'].astype(float)
blur = 5
infos['v'] = np.sqrt((infos['vx']**2+infos['vy']**2).astype(float))
infos['gd'] = np.sqrt((infos[f'grad_density_x{blur}']**2+infos[f'grad_density_y{blur}']**2).astype(float))
infos['spvgd']=(infos['vx']*infos[f'grad_density_x{blur}']+infos['vy']*infos[f'grad_density_y{blur}'])/(infos['v']*infos['gd'])
infos['angle_vgd']=get_angle(-infos[f'grad_density_x{blur}'],-infos[f'grad_density_y{blur}'],infos['vx'],infos['vy'])
infos['angle_Pgd']=infos['angle_vgd']+infos['angle_to_P']
infos['angle_Ngd']=infos['angle_vgd']+infos['angle_to_N']
corrected = infos.loc[(infos["straightness"] <= 1)& (infos["speed"] >=25)& (infos["speed"] <400)&(infos["straightness"] > 0.95)&(infos["density"]>0)]
baits = corrected.loc[corrected ['treatment']=='baits']
no_baits = corrected .loc[corrected ['treatment']=='25']

In [6]:
def get_density_map(plate,t,compress,blur,x,y,fz):
    densities=np.zeros((30000//compress,60000//compress),dtype=np.float)
    select = corrected.loc[(corrected['plate']==plate) & (corrected['t']==t)]
    for index,row in select.iterrows():
        xx = int(row[x])//compress
        yy = int(row[y])//compress
        densities[xx,yy]+=fz(row)
    density_filtered = gaussian_filter(densities,blur)
    return(density_filtered)

In [10]:
plates = set(corrected['plate'].values)
plates

{269,
 296,
 297,
 298,
 314,
 340,
 344,
 351,
 373,
 382,
 395,
 399,
 405,
 409,
 416,
 419,
 420,
 423,
 424,
 425,
 433,
 435,
 436}

In [12]:
for plate in plates:
    ts = set(corrected.loc[(corrected['plate']==419)]['t'])
    ts = list(ts)
    ts.sort()
    densities = [get_density_map(419,t,100,5,'x','y',lambda row : row['speed']) for t in ts]
    imageio.mimsave(f'Figure/speed_movie{plate}.gif', densities,duration = 1)

Lossy conversion from float64 to uint8. Range [0.0, 1.72143677708428]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3.957171889756781]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3.740454175547317]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3.322802597473528]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 2.498569646579737]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3.520259599237698]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 2.4290524530575817]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. R